In [72]:
import numpy as np
from scipy import linalg
from scipy import sparse
import scipy.sparse.linalg as spslinalg
import matplotlib.pyplot as plt
%matplotlib inline

import ed

In [75]:
L = 14
g = 1.5
J = 1.
sx_list = ed.gen_sx_list(L)
sz_list = ed.gen_sz_list(L)
H = ed.gen_hamiltonian(sx_list, sz_list, g, J)

k = 6
eigval, eigvec = spslinalg.eigsh(H, k, which='SA') # smallest algebraic eigevals
for i in range(1):
    print(np.around(eigval[i], 3), np.around(eigvec[:, i], 3))

ground = eigvec[:, 0]
assert ground.shape == (2**L, ) # throw error if false
print(np.inner(ground.conj(), ground), ' or ', np.linalg.norm(ground)) # <v0|v0> = 1
# print(np.inner(ground.conj(), H*ground)) # E0 = <v0|H|v0>

-23.223 [ 0.815 -0.    -0.    ...  0.     0.     0.   ]
1.0000000000000027  or  1.0000000000000013


In [186]:
def compress(psi, L, chimax):
    '''Compress a state into MPS form using successive SVDs.'''
    
    psi_aR = psi.reshape(1, 2**L) # add trivial bond
    M = [] # 3 indices (αn, jn, αn+1)
    
    for n in range(1, L+1):
        # R_n = (jn, jn+1, ..., jL) -> indices for remaining block of state psi
        chi_L, dim_R = psi_aR.shape
        assert dim_R == 2**(L-(n-1))
        print('chi_L:', chi_L, ' dim_R:', dim_R, end='')
        
        psi_LR = psi_aR.reshape(2*chi_L, dim_R//2) # 2*chi_L = L_n, dim_Rn//2 = dim_R_n+1
        M_n, lambda_n, psi_tilde = linalg.svd(psi_LR, full_matrices=False, lapack_driver='gesvd')
        
        # truncate to bond dimension chimax
        if len(lambda_n) > chimax:
            keep = np.argsort(lambda_n)[::-1][:chimax]
            M_n = M_n[:, keep]
            lambda_n = lambda_n[keep]
            psi_tilde = psi_tilde[keep, :]
    
        print('\t M_n:', M_n.shape, ' lambda_n:', lambda_n.shape,
              ' psi_tilde:', psi_tilde.shape, end='')
    
        chi_n_1 = len(lambda_n) # bond n+1 dimension
        M_n = M_n.reshape((chi_L, 2, chi_n_1)) # Ln=(αn, jn), (Ln, αn+1) -> (αn, jn, αn+1)
        M.append(M_n)
        
        # Reabsorb lambda_n into psi
        psi_aR = lambda_n[:, np.newaxis] * psi_tilde[:, :]
        print(' \t\tM_n:', M_n.shape, ' psi_aR:', psi_aR.shape)
        
    assert psi_aR.shape == (1, 1)
    print("Remaining psi_aR in compress: ", psi_aR)
    return M

In [187]:
M = compress(ground, L, 2**(L//2))

chi_L: 1  dim_R: 16384	 M_n: (2, 2)  lambda_n: (2,)  psi_tilde: (2, 8192) 		M_n: (1, 2, 2)  psi_aR: (2, 8192)
chi_L: 2  dim_R: 8192	 M_n: (4, 4)  lambda_n: (4,)  psi_tilde: (4, 4096) 		M_n: (2, 2, 4)  psi_aR: (4, 4096)
chi_L: 4  dim_R: 4096	 M_n: (8, 8)  lambda_n: (8,)  psi_tilde: (8, 2048) 		M_n: (4, 2, 8)  psi_aR: (8, 2048)
chi_L: 8  dim_R: 2048	 M_n: (16, 16)  lambda_n: (16,)  psi_tilde: (16, 1024) 		M_n: (8, 2, 16)  psi_aR: (16, 1024)
chi_L: 16  dim_R: 1024	 M_n: (32, 32)  lambda_n: (32,)  psi_tilde: (32, 512) 		M_n: (16, 2, 32)  psi_aR: (32, 512)
chi_L: 32  dim_R: 512	 M_n: (64, 64)  lambda_n: (64,)  psi_tilde: (64, 256) 		M_n: (32, 2, 64)  psi_aR: (64, 256)
chi_L: 64  dim_R: 256	 M_n: (128, 128)  lambda_n: (128,)  psi_tilde: (128, 128) 		M_n: (64, 2, 128)  psi_aR: (128, 128)
chi_L: 128  dim_R: 128	 M_n: (256, 64)  lambda_n: (64,)  psi_tilde: (64, 64) 		M_n: (128, 2, 64)  psi_aR: (64, 64)
chi_L: 64  dim_R: 64	 M_n: (128, 32)  lambda_n: (32,)  psi_tilde: (32, 32) 		M_n: (64, 2, 32)